In [47]:
import numpy as np
import fractions

np.set_printoptions(formatter={'all':lambda x: str(fractions.Fraction(x).limit_denominator())}) # display as fractions (1/3)
# np.set_printoptions(formatter={'all':lambda x: str(x)}) # display as decimal numbers (0.33333...)

# Lineare Regression

In [48]:
# Datenmatrix X und Vektor y der Zielgrössen

X = np.array([
    [1, -5],
    [1, -4],
    [1, 2],
    [1, 6],
    [1, 7],
])

y = np.array([
    [-2],
    [2],
    [3],
    [5],
    [6],
])

## "$ X^TX $"

In [49]:
X.T @ X

array([[5, 6],
       [6, 130]])

## Vektor der Regressionskoeffizienten "$ \hat{b} = (X^TX)^{-1}X^Ty $"

In [50]:
bhat = np.linalg.inv(X.T @ X) @ X.T @ y
bhat

array([[670/307],
       [158/307]])

## Projektionsmatrix "$ P = X(X^TX)^{-1}X^T $"

In [51]:
P = X @ np.linalg.inv(X.T @ X) @ X.T
P

array([[315/614, 142/307, 49/307, -13/307, -57/614],
       [142/307, 129/307, 51/307, -1/307, -14/307],
       [49/307, 51/307, 63/307, 71/307, 73/307],
       [-13/307, -1/307, 71/307, 119/307, 131/307],
       [-57/614, -14/307, 73/307, 131/307, 291/614]])

## Residualmatrix "$ Q = I - P $"

In [52]:
I = np.identity(P.shape[0])
Q = I - P
Q

array([[299/614, -142/307, -49/307, 13/307, 57/614],
       [-142/307, 178/307, -51/307, 1/307, 14/307],
       [-49/307, -51/307, 244/307, -71/307, -73/307],
       [13/307, 1/307, -71/307, 188/307, -131/307],
       [57/614, 14/307, -73/307, -131/307, 323/614]])

## Prädiktionsvektor "$ \hat{y} = Py $"

In [53]:
yhat = P @ y
yhat

array([[-120/307],
       [38/307],
       [986/307],
       [1618/307],
       [1776/307]])

## Fehlervektor "$ \mathbf{e} = y - \hat{y} = Qy $"

In [54]:
# entweder
e = y - yhat

# oder
e = Q @ y

e

array([[-494/307],
       [576/307],
       [-65/307],
       [-83/307],
       [66/307]])

## SQR (Summe der Quadrate der Restabweichungen) "$ e^Te $"

In [55]:
SQR = e.T @ e
SQR

array([[1926/307]])

## SQT (Summe der Quadrate der Totalen Abweichungen)

### Zentrierende Matrix "$ M = I - \frac{1}{m}\mathbf{1}\mathbf{1}^T $"

In [56]:
m, _ = y.shape
M = np.eye (m) - np.ones ((m, m)) / m
print(f"m={m}")
M

m=5


array([[4/5, -1/5, -1/5, -1/5, -1/5],
       [-1/5, 4/5, -1/5, -1/5, -1/5],
       [-1/5, -1/5, 4/5, -1/5, -1/5],
       [-1/5, -1/5, -1/5, 4/5, -1/5],
       [-1/5, -1/5, -1/5, -1/5, 4/5]])

## SQT ausrechnen "$ y^TMy $"

In [57]:
SQT = y.T @ M @ y
SQT

array([[194/5]])

## Bestimmtheitsmass "$ R^2 = 1 - \frac{SQR}{SQT} $"

In [58]:
R2 = 1 - SQR/SQT
R2

array([[24964/29779]])

# Hauptkomponentenanalyse

In [59]:
# Datenmatrix x-tilde
Xtilde = np.array([
    [1, 2],
    [3, -1],
    [5, -2],
    [8, -3],
    [9, -5],
])

## Zentrierende Datenmatrix "$ M = I - \frac{1}{m}\mathbf{1}\mathbf{1}^T $"

In [60]:
m, _ = Xtilde.shape
M = np.eye (m) - np.ones ((m, m)) / m
print(f"m={m}")
M

m=5


array([[4/5, -1/5, -1/5, -1/5, -1/5],
       [-1/5, 4/5, -1/5, -1/5, -1/5],
       [-1/5, -1/5, 4/5, -1/5, -1/5],
       [-1/5, -1/5, -1/5, 4/5, -1/5],
       [-1/5, -1/5, -1/5, -1/5, 4/5]])

## Zentrierte Datenmatrix "$ X = M\tilde{X} $"

In [61]:
X = M @ Xtilde
X

array([[-21/5, 19/5],
       [-11/5, 4/5],
       [-1/5, -1/5],
       [14/5, -6/5],
       [19/5, -16/5]])

## Kovarianzmatrix "$ C = \frac{1}{m-1}X^TX $"

In [62]:
C = (1 / (m-1)) * X.T @ X
C

array([[56/5, -83/10],
       [-83/10, 67/10]])

## Eigenwertzerlegung / Spektralzerlegung der Kovarianzmatrix "$ \mathbf{C} $"

## Eigenwerte und Eigenvektoren berechnen

In [63]:
ew, ev = np.linalg.eig(C)
print(ew)
print()
print(ev)

[2537509/144591 198139/565407]

[[780551/982762 446587/735000]
 [-446587/735000 780551/982762]]


### Matrizen zusammensetzen: $ \mathbf{C}=\mathbf{V}\Lambda\mathbf{V}^\intercal $

- $ \mathbf{V} $: Spalten = Eigenvektoren
- $ \Lambda $: Diagonalmatrix mit Eigenwerten
- $ \mathbf{V}^\intercal $: Zeilen = Eigenvektoren

## Singulärwertzerlegung der zentrierten Datenmatrix $ \mathbf{X} $

In [64]:
U, D, VT = np.linalg.svd(X, full_matrices=False)
Sigma = np.diag (D)

print(U)
print()
print(Sigma)
print()
print(VT)

[[-269357/399807 214508/544767]
 [-242511/909757 -353224/596297]
 [-4021/902528 -165045/696959]
 [164174/465805 420423/665284]
 [467512/789333 -90049/458180]]

[[3616009/431585 0]
 [0 998914/843711]]

[[780551/982762 -446587/735000]
 [446587/735000 780551/982762]]


# Eigenwerte / Eigenvektoren

In [65]:
A = np.array([
    [3, 7],
    [7, 2],
])

np.linalg.eig(A)

(array([6082058/639017, -2886973/639017]),
 array([[724289/989651, -601681/882940],
        [601681/882940, 724289/989651]]))

# Projektion auf einen Vektor (eindimensionaler Teilraum)

In [66]:
# Aufgabenstellung: Projizieren Sie den Vektor b auf eine Gerade durch a.

b = np.array([
    [3],
    [4],
    [4],
])

a = np.array([
    [2],
    [2],
    [1],
])

## "$ \mathbf{a}^\intercal\mathbf{a} $"

In [67]:
ata = a.T @ a
ata

array([[9]])

## Projektionsmatrix "$ P = \frac{1}{a^Ta} $"

In [68]:
P = (1 / ata) * a @ a.T
P

array([[4/9, 4/9, 2/9],
       [4/9, 4/9, 2/9],
       [2/9, 2/9, 1/9]])

## Projektion von $ \mathbf{b} $ "$ Pb $"

In [69]:
Pb = P @ b
Pb

array([[4],
       [4],
       [2]])

## Fehlervektor "$ e = b -Pb $"

In [70]:
e = b - Pb
e

array([[-1],
       [0],
       [2]])

## Check ob Fehlervektor orthogonal zu "$ \mathbf{a} $" ist

In [71]:
# falls 0 => orthogonal
a.T @ e

array([[0]])

# Projektion auf einen mehrdimensionalen Teilraum

In [72]:
# Aufgabenstellung: Projizieren Sie den Vektor v auf die Ebene A

b = np.array([
    [3],
    [4],
    [4],
])

A = np.array([
    [2, 1],
    [2, 0],
    [1, 0],
])

## "$ \mathbf{A}^\intercal\mathbf{A} $"

In [73]:
ATA = A.T @ A
ATA

array([[9, 2],
       [2, 1]])

## Projektionsmatrix "$ P = A(A^TA)^{-1}A^T $"

In [75]:
P = A @ np.linalg.inv(ATA) @ A.T
P

array([[1, 0, 0],
       [0, 4/5, 2/5],
       [0, 2/5, 1/5]])

## Projektion von $ \mathbf{b} $ "$ Pb $"

In [76]:
Pb = P @ b
Pb

array([[3],
       [24/5],
       [12/5]])

## Fehlervektor "$ \mathbf{e} $"

In [77]:
e = b - Pb
e

array([[0],
       [-4/5],
       [8/5]])

## Check ob Fehlervektor orthogonal zu den Spaltenvektoren von "$ \mathbf{A} $" ist

In [81]:
a1 = A[:, 0]
a2 = A[:, 1]

array([1, 0, 0])

In [82]:
# falls 0 => orthogonal
a1.T @ e

array([0])

In [83]:
# falls 0 => orthogonal
a2.T @ e

array([0])

# Schnittwinkel zweier Geraden

$ \cos{\alpha} = \frac{\vec{u} * \vec{v}}{\lvert \vec{u} \rvert * \lvert \vec{v} \rvert}$

In [89]:
u = np.array([4, 2, -1])  # Richtungsvektor erste Gerade
v = np.array([5, -2, 3])  # Richtungsvektor zweite Gerade

c = np.dot(u, v) / np.linalg.norm(u) / np.linalg.norm(v)  # Kosinus des Winkels
angle_rad = np.arccos(np.clip(c, -1, 1)) # Winkel in rad
angle_deg = np.rad2deg(angle_rad) # Winkel in grad
angle_deg


62.60030804589705